In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python",
      "version": "3.12.0",
      "mimetype": "text/x-python",
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "pygments_lexer": "ipython3",
      "nbconvert_exporter": "python",
      "file_extension": ".py"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Proprietary content. © Great Learning. All Rights Reserved. Unauthorized use or distribution"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Python Version"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import sys\n",
        "print(f'Python Version: {sys.version}')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Learning Objectives\n",
        "\n",
        "1. Understand the parameters available to control text generation from LLM APIs.\n",
        "\n",
        "2. Implement key prompt engineering patterns, that is, few-shot, chain-of-thought, rephrase & respond, self-consistency, tree-of-thought & LLM-as-a-judge.\n",
        "\n",
        "# Setup\n",
        "\n",
        "Azure provides seamless integration with the OpenAI API to provide access to the GPT-series of models."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# Install required packages\n",
        "!pip install \\\n",
        "  \"openai~=1.107.0\" \\\n",
        "  \"requests~=2.32.4\" \\\n",
        "  \"tqdm~=4.67.1\" \\\n",
        "  \"packaging~=25.0\" \\\n",
        "  \"typing_extensions~=4.15.0\" \\\n",
        "  \"pydantic~=2.11.7\""
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "from openai import AzureOpenAI\n",
        "import os\n",
        "from getpass import getpass\n",
        "\n",
        "# Option 1: Use environment variables (recommended)\n",
        "# Set these in your environment before running:\n",
        "# export AZURE_API_KEY=\"your-api-key\"\n",
        "# export AZURE_ENDPOINT=\"your-endpoint\"\n",
        "\n",
        "# Option 2: Enter manually (for testing)\n",
        "azure_api_key = os.getenv('AZURE_API_KEY') or getpass('Enter your Azure API Key: ')\n",
        "azure_endpoint = os.getenv('AZURE_ENDPOINT') or input('Enter your Azure Endpoint: ')\n",
        "\n",
        "# If you prefer to hardcode (not recommended for production):\n",
        "# azure_api_key = \"your-api-key-here\"\n",
        "# azure_endpoint = \"https://your-resource.openai.azure.com/\"\n",
        "\n",
        "api_version = \"2024-06-01\"\n",
        "\n",
        "client = AzureOpenAI(\n",
        "  azure_endpoint = azure_endpoint,\n",
        "  api_key=azure_api_key,\n",
        "  api_version=api_version\n",
        ")\n",
        "\n",
        "model_name = 'gpt-4o-mini' # deployment name\n",
        "print(\"Azure OpenAI client initialized successfully!\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Prompt Structure\n",
        "\n",
        "Prompts presented to the Azure Open AI API for inference have to follow a specific structure with three roles - `system/developer`, `user` and `assistant`.\n",
        "\n",
        "These roles are:\n",
        "\n",
        "- `system/developer`: A set of instructions specified for the LLM to follow, as outlined by the application developer. The Open AI API considers the terms `system` and `developer` interchangeable, with `developer` being the preferred term for future use. Going forward Open AI will reserve the `system` keyword for additional instructions Open AI might insert into the prompt for better performance (as of now this is not a rigid rule).\n",
        "- `user`: A placeholder for users (i.e., end-users of the application) to present their input\n",
        "- `assistant`: Response from the LLM where the system message is applied to the user input.\n",
        "\n",
        "LLMs are tuned to understand sets of instructions as defined by these roles. LLM APIs provide a mechanism to encapsulate the *constant* portion of these instructions as the *system prompt*. While it is is optional, when a system prompt (e.g., \"Classify the sentiment of the input sentence. Do not answer any other question\") is mentioned, it is automatically pasted ahead of all the instructions entered by the user without us needing to explictly append it with every instruction.\n",
        "\n",
        "System/Developer messages are a great way to restrict the behaviour of the LLM to a specific, controlled set of instructions. Since end-users of the application have no access to the developer message (and can be edited by only the application developer), there is very less chance of the application being hijacked beyond its intended purpose.\n",
        "\n",
        "The `user` and `assistant` roles enable:\n",
        "- multi-turn conversations\n",
        "- showcasing ideal responses expected from the model\n",
        "\n",
        "A typical prompt structure is presented in the figure below.\n",
        "\n",
        "Each example is a pair of `user` and `assistant` messages that illustrates expected output from the LLM.\n",
        "\n",
        "Let us see these roles in action."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "system_message = \"\"\"\n",
        "You are an assistant to a hospital administration team working on extracting important information from medical notes made by doctors.\n",
        "Medical notes will be presented to you in the user input.\n",
        "Extract relevant information as mentioned below in a json format with the following schema.\n",
        "- age: integer, age of the patient\n",
        "- gender: string, can be one of male, female or other\n",
        "- diagnosis: string, can be one of migraine, diabetes, arthritis and acne\n",
        "- weight: integer, weight of the patient\n",
        "- smoking: string, can be one of yes or no\n",
        "\"\"\"\n",
        "\n",
        "user_input = \"\"\"\n",
        "Medical Notes:\n",
        "---\n",
        "A 35-year-old male patient, Mr. Nags, presented with symptoms\n",
        "of increased thirst, frequent urination, fatigue, and unexplained\n",
        "weight loss. Upon evaluation, he was diagnosed with diabetes,\n",
        "confirmed by elevated blood sugar levels. Mr. Nags' weight\n",
        "is 80 kgs. He has been prescribed Metformin to be taken twice daily\n",
        "with meals. It was noted during the consultation that the patient is\n",
        "a current smoker.\n",
        "\"\"\"\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=[\n",
        "        {\"role\": \"developer\", \"content\": system_message},\n",
        "        {\"role\": \"user\", \"content\": user_input}\n",
        "    ]\n",
        ")\n",
        "\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "user_input = \"\"\"\n",
        "Medical Notes:\n",
        "---\n",
        "Patient Name: Ms. Krishnaveni\n",
        "Age: 45 years\n",
        "Gender: Female\n",
        "\n",
        "Chief Complaint:\n",
        "Ms. Krishnaveni presented with complaints of persistent abdominal pain, bloating, and changes in bowel habits over the past two months.\n",
        "\n",
        "History of Present Illness:\n",
        "Ms. Krishnaveni reports experiencing intermittent abdominal pain, predominantly in the lower abdomen, accompanied by bloating and alternating episodes of diarrhea and constipation. She describes the pain as crampy in nature, relieved partially by defecation but worsening after meals. There is no association with specific food items. She denies any rectal bleeding, unintended weight loss, or fever.\n",
        "\n",
        "Past Medical History:\n",
        "Ms. Krishnaveni has a history of irritable bowel syndrome (IBS), diagnosed five years ago, managed with dietary modifications and occasional use of over-the-counter antispasmodics.\n",
        "\n",
        "Medications:\n",
        "She occasionally takes over-the-counter antispasmodics for symptomatic relief of abdominal discomfort related to IBS.\n",
        "\n",
        "Family History:\n",
        "There is no significant family history of gastrointestinal disorders or malignancies.\n",
        "\n",
        "Social History:\n",
        "Ms. Krishnaveni is a non-smoker and does not consume alcohol. She works as a teacher in a local school.\n",
        "\"\"\"\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=[\n",
        "        {\"role\": \"developer\", \"content\": system_message},\n",
        "        {\"role\": \"user\", \"content\": user_input}\n",
        "    ]\n",
        ")\n",
        "\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Prompt Parameters\n",
        "\n",
        "## Maximum tokens\n",
        "\n",
        "The parameter (`max_tokens`) refers to the maximum number of tokens that can be generated in the chat completion. With this parameter, we can modify the output length like so:"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "system_message = \"\"\"\n",
        "You are an assistant to the marketing team for the gaming company Razer tasked to create advertising content for the company.\n",
        "You will be presented with product information in the input.\n",
        "With this information, write a sleek \"About this item\" description that will be used on its Amazon product page.\n",
        "Use bullet points to delineate key features mentioned in the description.\n",
        "\"\"\"\n",
        "\n",
        "user_input = \"\"\"\n",
        "Below is the metadata about the Razer Ornata V3 X gaming keyboard:\n",
        "Brand: Razer\n",
        "Series: Ornata V3 X\n",
        "Item model number: RZ03-04470200-R3U1\n",
        "Hardware Platform: PC\n",
        "Operating System: Microsoft Windows\n",
        "Item Weight: 2.97 pounds\n",
        "Product Dimensions: 17.46 x 5.68 x 1.23 inches\n",
        "Item Dimensions LxWxH: 17.46 x 5.68 x 1.23 inches\n",
        "Color: Classic Black\n",
        "Manufacturer: Razer\n",
        "Language: English\n",
        "ASIN: B09X6GJ691\n",
        "Special Features: Low-Profile Keys, Spill Resistant, Ergonomic Wrist Rest, Chroma RGB Lighting, Silent Membrane Switches, Cable Routing Options\n",
        "\"\"\"\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=[\n",
        "        {\"role\": \"developer\", \"content\": system_message},\n",
        "        {\"role\": \"user\", \"content\": user_input}\n",
        "    ],\n",
        "    max_tokens=1024\n",
        ")\n",
        "\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Temperature"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "system_message = \"\"\"\n",
        "You are an assistant to the marketing team for the gaming company Razer tasked to create advertising content for the company.\n",
        "You will be presented with product information in the input.\n",
        "With this information, write a sleek \"About this item\" description that will be used on its Amazon product page.\n",
        "Use bullet points to delineate key features mentioned in the description.\n",
        "\"\"\"\n",
        "\n",
        "user_input = \"\"\"\n",
        "Below is the metadata about the Razer Ornata V3 X gaming keyboard:\n",
        "Brand: Razer\n",
        "Series: Ornata V3 X\n",
        "Item model number: RZ03-04470200-R3U1\n",
        "Hardware Platform: PC\n",
        "Operating System: Microsoft Windows\n",
        "Item Weight: 2.97 pounds\n",
        "Product Dimensions: 17.46 x 5.68 x 1.23 inches\n",
        "Item Dimensions LxWxH: 17.46 x 5.68 x 1.23 inches\n",
        "Color: Classic Black\n",
        "Manufacturer: Razer\n",
        "Language: English\n",
        "ASIN: B09X6GJ691\n",
        "Special Features: Low-Profile Keys, Spill Resistant, Ergonomic Wrist Rest, Chroma RGB Lighting, Silent Membrane Switches, Cable Routing Options\n",
        "\"\"\"\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=[\n",
        "        {\"role\": \"developer\", \"content\": system_message},\n",
        "        {\"role\": \"user\", \"content\": user_input}\n",
        "    ],\n",
        "    temperature=0.4,\n",
        "    max_tokens=256,\n",
        "    n=2\n",
        ")\n",
        "\n",
        "print(\"Response 1:\")\n",
        "print(response.choices[0].message.content)\n",
        "print(\"\\n\" + \"=\"*50 + \"\\n\")\n",
        "print(\"Response 2:\")\n",
        "print(response.choices[1].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "Reducing the temperature reduces variability in generation."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=[\n",
        "        {\"role\": \"developer\", \"content\": system_message},\n",
        "        {\"role\": \"user\", \"content\": user_input}\n",
        "    ],\n",
        "    temperature=0,\n",
        "    max_tokens=256,\n",
        "    n=2\n",
        ")\n",
        "\n",
        "print(\"Response 1:\")\n",
        "print(response.choices[0].message.content)\n",
        "print(\"\\n\" + \"=\"*50 + \"\\n\")\n",
        "print(\"Response 2:\")\n",
        "print(response.choices[1].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Structuring Prompts\n",
        "\n",
        "## Few-shot prompting\n",
        "\n",
        "While system/developer messages could be used to control the behaviour of LLMs, they become quickly unwieldy when we expect the output to follow a specific format (e.g., JSON). In such situations, few examples go a long way in specifying the behavior of the LLM (i.e., *show, rather than tell*). This technique is referred to as few-shot prompting.\n",
        "\n",
        "Few shot prompt relies on assembling exemplars that specify the output format from the LLM. These exemplars could represent text-to-label tasks or text-to-text tasks.\n",
        "\n",
        "Remember, that these examples are only for illustration of format; the LLM is in inference mode and does not adapt its internal representation based on these examples.\n",
        "\n",
        "Let us now implement a few-shot prompt in code. While we will task the LLM to execute sentiment analysis, we will force the model to follow a specific output format. Instead of describing the format in a system message, we will show the format in action as a set of two examples of assistant responses."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "few_shot_system_message = \"\"\"\n",
        "You are a product marketer tasked to classify customer reviews in the input as positive or negative in sentiment.\n",
        "Do not explain your answer.\n",
        "Present your output in the following format:\n",
        "{'sentiment': <'positive' or 'negative'>}\n",
        "\"\"\"\n",
        "\n",
        "user_input_example1 = \"\"\"\n",
        "Review:\n",
        "I couldn't be happier with my experience at your store!\n",
        "The staff went above and beyond to assist me, providing exceptional customer service.\n",
        "They were friendly, knowledgeable, and genuinely eager to help.\n",
        "The product I purchased exceeded my expectations and was exactly what I was looking for.\n",
        "From start to finish, everything was seamless and enjoyable.\n",
        "I will definitely be returning and recommending your store to all my friends and family.\n",
        "Thank you for making my shopping experience so wonderful!\n",
        "\"\"\"\n",
        "\n",
        "assistant_output_example1 = \"{'sentiment': 'positive'}\"\n",
        "\n",
        "user_input_example2 = \"\"\"\n",
        "Review:\n",
        "I am extremely disappointed with the service I received at your store!\n",
        "The staff was rude and unhelpful, showing no regard for my concerns.\n",
        "Not only did they ignore my requests for assistance, but they also had the audacity to speak to me condescendingly.\n",
        "It's clear that your company values profit over customer satisfaction.\n",
        "I will never shop here again and will make sure to spread the word about my awful experience.\n",
        "You've lost a loyal customer, and I hope others steer clear of your establishment!\n",
        "\"\"\"\n",
        "\n",
        "assistant_output_example2 = \"{'sentiment': 'negative'}\"\n",
        "\n",
        "new_user_input = \"\"\"\n",
        "Review:\n",
        "The layout of the store was well-thought-out, with clear signage and organized aisles that made it easy to navigate.\n",
        "I appreciated the strategic placement of product categories, which not only facilitated a smooth shopping experience but also made it effortless to find exactly what I was looking for.\n",
        "The store's cleanliness and neat displays added to the overall appeal, creating an aesthetically pleasing environment.\n",
        "\"\"\"\n",
        "\n",
        "few_shot_prompt = [\n",
        "        {\"role\": \"developer\", \"content\": few_shot_system_message},\n",
        "        {\"role\": \"user\", \"content\": user_input_example1},\n",
        "        {\"role\": \"assistant\", \"content\": assistant_output_example1},\n",
        "        {\"role\": \"user\", \"content\": user_input_example2},\n",
        "        {\"role\": \"assistant\", \"content\": assistant_output_example2},\n",
        "        {\"role\": \"user\", \"content\": new_user_input}\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=few_shot_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "To reiterate, the model does not \"learn\" from the content of the examples. It simply learns the format of the input and output. To verify this, let us swap labels of the examples."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "few_shot_prompt = [\n",
        "        {\"role\": \"developer\", \"content\": few_shot_system_message},\n",
        "        {\"role\": \"user\", \"content\": user_input_example1},\n",
        "        {\"role\": \"assistant\", \"content\": assistant_output_example2},\n",
        "        {\"role\": \"user\", \"content\": user_input_example2},\n",
        "        {\"role\": \"assistant\", \"content\": assistant_output_example1},\n",
        "        {\"role\": \"user\", \"content\": new_user_input}\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=few_shot_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "As the above output indicates, the model does not change its answer.\n",
        "\n",
        "Finally, a simpler variant of few-shot prompting where no examples are provided is called zero-shot prompting. Here is a zero-shot prompt in action."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "zero_shot_system_message = \"\"\"\n",
        "You are a product marketer tasked to classify customer reviews in the input as positive or negative in sentiment.\n",
        "Do not explain your answer.\n",
        "Present your output in the following format:\n",
        "{'sentiment': <'positive' or 'negative'>}\n",
        "\"\"\"\n",
        "\n",
        "new_user_input = \"\"\"\n",
        "Review:\n",
        "The layout of the store was well-thought-out, with clear signage and organized aisles that made it easy to navigate.\n",
        "I appreciated the strategic placement of product categories, which not only facilitated a smooth shopping experience but also made it effortless to find exactly what I was looking for.\n",
        "The store's cleanliness and neat displays added to the overall appeal, creating an aesthetically pleasing environment.\n",
        "\"\"\"\n",
        "\n",
        "zero_shot_prompt = [\n",
        "        {\"role\": \"developer\", \"content\": zero_shot_system_message},\n",
        "        {\"role\": \"user\", \"content\": new_user_input}\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=zero_shot_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Chain-of-Thought (CoT) prompting\n",
        "\n",
        "Chain-of-Thought prompting is a technique used in generative AI tasks to guide the model's response generation by providing a sequence of related prompts or questions. Instead of a single prompt, a CoT consists of multiple interconnected steps that build upon each other to guide the model's thinking process. These steps represent the \"thinking\" process that we want the model to follow.\n",
        "\n",
        "The purpose of CoT prompting is to encourage the model to generate more coherent and contextually relevant responses by guiding its thought process in a structured manner. Each step in the chain serves as a stepping stone, providing additional context or constraints for the model to consider while generating the response.\n",
        "\n",
        "CoT prompts could also be augmented with few-shot examples, so that the prompt guides the reasoning power of the model while examples guide expected output.\n",
        "\n",
        "Let us now implement a chain-of-thought prompt for entity extraction. Let us begin by writing a system message that outlines clearly the expected"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "system_message = \"\"\"\n",
        "You are an assistant that helps a customer service representatives from a mobile phone company to better understand customer complaints.\n",
        "For each complaint, extract the following information and present it only in a JSON format:\n",
        "1. phone_model: This is the name of the phone - if unknown, just say "UNKNOWN"\n",
        "2. phone_price: The price in dollars - if unknown, assume it to be 1000 $\n",
        "3. complaint_desc: A short description/summary of the complaint in less than 20 words\n",
        "4. additional_charges: How much in dollars did the customer spend to fix the problem? - this should be an integer\n",
        "5. refund_expected: TRUE or FALSE - check if the customer explicitly mentioned the word "refund" to tag as TRUE. If unknown, assume that the customer is not expecting a refund\n",
        "\n",
        "Take a step-by-step approach in your response, before sharing your final answer in the following JSON format:\n",
        "{\n",
        "    phone_model: ,\n",
        "    phone_price: ,\n",
        "    complaint_desc: ,\n",
        "    additional_charges: ,\n",
        "    refund_expected: \n",
        "}\n",
        "\n",
        "Explain your reasoning before presenting the final answer.\n",
        "\"\"\"\n",
        "\n",
        "customer_complaint = \"\"\"\n",
        "I am fuming with anger and regret over my purchase of the XUI890.\n",
        "First, the price tag itself was exorbitant at 1500 $, making me expect exceptional quality.\n",
        "Instead, it turned out to be a colossal disappointment.\n",
        "The additional charges to fix its constant glitches and defects drained my wallet even more.\n",
        "I spend 275 $ to get a new battery.\n",
        "The final straw was when the phone's camera malfunctioned, and the repair cost was astronomical.\n",
        "I demand a full refund and an apology for this abysmal product.\n",
        "Returning it would be a relief, as this phone has become nothing but a money pit. Beware, fellow buyers!\n",
        "\"\"\"\n",
        "\n",
        "system_message1 = \"\"\"\n",
        "You are an assistant that helps a customer service representatives from a mobile phone company to better understand customer feedback.\n",
        "Analyze the customer feedback presented in the input and extract key sentiments and themes.\n",
        "For each piece of feedback:\n",
        "1. Identify the main sentiment (positive, negative, neutral)\n",
        "2. Extract key themes or topics mentioned\n",
        "3. Note any specific features or products mentioned\n",
        "\"\"\"\n",
        "\n",
        "first_stage_prompt = [\n",
        "    {\"role\": \"developer\", \"content\": system_message1},\n",
        "    {\"role\": \"user\", \"content\": customer_complaint}\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=first_stage_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "first_stage_output = response.choices[0].message.content\n",
        "\n",
        "print(\"First Stage Output:\")\n",
        "print(first_stage_output)\n",
        "print(\"\\n\" + \"=\"*50 + \"\\n\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "system_message2 = \"\"\"\n",
        "You are an assistant that helps a customer service representatives from a mobile phone company to better understand customer complaints.\n",
        "Based on the structured feedback analysis presented in the input, generate actionable insights and recommendations.\n",
        "For each theme identified:\n",
        "1. Summarize the overall sentiment\n",
        "2. List specific issues or strengths mentioned\n",
        "3. Provide concrete recommendations for improvement\n",
        "4. Prioritize actions based on impact and effort\n",
        "\"\"\"\n",
        "\n",
        "second_stage_prompt = [\n",
        "    {\"role\": \"developer\", \"content\": system_message2},\n",
        "    {\"role\": \"user\", \"content\": first_stage_output}\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=second_stage_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "final_output = response.choices[0].message.content\n",
        "\n",
        "print(\"Final Output:\")\n",
        "print(final_output)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Self-consistency\n",
        "\n",
        "In self-consistency, we generate multiple answers to the same question and pick the answer that is repeated the most across these occurrences. This is particularly valuable for factual questions."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "generation_system_message = \"\"\"\n",
        "You are a helpful assistant tasked to answer queries on financial information.\n",
        "The context needed to answer these queries is presented below:\n",
        "\n",
        "---\n",
        "Context:\n",
        "In 2022, we recognized total revenues of $81.46 billion, respectively, representing an increase of $27.64 billion, compared to the prior year.\n",
        "We continue to ramp production, build new manufacturing capacity and expand our operations to enable increased deliveries and deployments of our products and further revenue growth.\n",
        "---\n",
        "\n",
        "You will be presented a question in the input.\n",
        "Using the context above generate 3 distinct answers to the question.\n",
        "Arrange your answers in numbered bullet points.\n",
        "\"\"\"\n",
        "\n",
        "user_message_template = \"\"\"\n",
        "Question:\n",
        "{question}.\n",
        "\"\"\"\n",
        "\n",
        "factual_question = \"What was the increase in annual revenue in 2022 compared to 2021?\"\n",
        "\n",
        "answers_prompt = [\n",
        "    {'role':'developer', 'content': generation_system_message},\n",
        "     {'role': 'user', 'content': user_message_template.format(\n",
        "         question=factual_question\n",
        "         )\n",
        "     }\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=answers_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "factual_answers = response.choices[0].message.content\n",
        "\n",
        "print(\"Generated Answers:\")\n",
        "print(factual_answers)\n",
        "print(\"\\n\" + \"=\"*50 + \"\\n\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "consistency_system_message = \"\"\"\n",
        "You are a helpful assistant tasked to answer queries on financial information.\n",
        "You will be presented a question and 3 AI generated answers to the question in the user input.\n",
        "Observe the answers mentioned in the input and choose the answer that occurs most.\n",
        "Present only the most frequent solution in the following format.\n",
        "Final Answer:\n",
        "\"\"\"\n",
        "\n",
        "consistency_user_message = \"\"\"\n",
        "Question:\n",
        "{question}.\n",
        "Answers:\n",
        "{answers}.\n",
        "\"\"\"\n",
        "\n",
        "consistency_prompt = [\n",
        "    {'role':'developer', 'content': consistency_system_message},\n",
        "     {'role': 'user', 'content': consistency_user_message.format(\n",
        "         question=factual_question,\n",
        "         answers=factual_answers\n",
        "         )\n",
        "     }\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=consistency_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "print(\"Self-Consistency Result:\")\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Rephrase and Respond"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "rephrase_system_message = \"\"\"\n",
        "You are a helpful assistant tasked to answer queries on financial information.\n",
        "\n",
        "The context required for you to answer the queries is presented below:\n",
        "\n",
        "---\n",
        "Context:\n",
        "In 2022, we recognized total revenues of $81.46 billion, respectively, representing an increase of $27.64 billion, compared to the prior year.\n",
        "We continue to ramp production, build new manufacturing capacity and expand our operations to enable increased deliveries and deployments of our products and further revenue growth.\n",
        "---\n",
        "\n",
        "You will be presented with a question in the input.\n",
        "Using the context presented above, rephrase and expand the question to help you do better answering.\n",
        "Maintain all the information in the original question.\n",
        "Please note that you only have to rephrase the question, do not mention the context or answer the question at this point.\n",
        "The context is only presented for your reference.\n",
        "\"\"\"\n",
        "\n",
        "user_message = \"\"\"\n",
        "Question:\n",
        "What was the increase in annual revenue in 2022 compared to 2021?\n",
        "\"\"\"\n",
        "\n",
        "rephrase_prompt = [\n",
        "    {'role':'developer', 'content': rephrase_system_message},\n",
        "    {'role': 'user', 'content': user_message}\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=rephrase_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "rephrased_factual_question = response.choices[0].message.content\n",
        "\n",
        "print(\"Rephrased Question:\")\n",
        "print(rephrased_factual_question)\n",
        "print(\"\\n\" + \"=\"*50 + \"\\n\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "respond_system_message = \"\"\"\n",
        "You are a helpful assistant tasked to answer queries on financial information.\n",
        "\n",
        "The context required for you to answer the queries is presented below:\n",
        "\n",
        "---\n",
        "Context:\n",
        "In 2022, we recognized total revenues of $81.46 billion, respectively, representing an increase of $27.64 billion, compared to the prior year.\n",
        "We continue to ramp production, build new manufacturing capacity and expand our operations to enable increased deliveries and deployments of our products and further revenue growth.\n",
        "---\n",
        "\n",
        "You will be presented with a question and a rephrased variant in the input.\n",
        "Using the context presented above, use your answer for the rephrased question presented above to answer the original question.\n",
        "Present your final answer in the following format.\n",
        "Final Answer:\n",
        "\"\"\"\n",
        "\n",
        "user_message = \"\"\"\n",
        "Original Question:\n",
        "What was the increase in annual revenue in 2022 compared to 2021?\n",
        "\n",
        "Rephrased Question:\n",
        "{}\n",
        "\"\"\".format(rephrased_factual_question)\n",
        "\n",
        "response_prompt = [\n",
        "    {'role':'developer', 'content': respond_system_message},\n",
        "    {'role': 'user', 'content': user_message}\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=response_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "print(\"Final Answer:\")\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## LLM-as-a-Judge\n",
        "\n",
        "In this method, we use another LLM to rate the performance of the LLM used in the original task (see figure below for an example in case of summarization). This method of using LLMs to evaluate LLMs is usualy referred to as LLM-as-a-judge. When LLMs are used to evaluate output, the system message should clearly define the rubrics used for evaluation and the key aspects of the output that should be evaluated. The advantage of using LLMs as judges is that we do not need human baselines (that are costly to collect), while writing down the rubrics for assessment is usually an easier task."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "example_dialogue = \"\"\"\n",
        "Dialogue:\n",
        "#Person1#: Excuse me, could you tell me where physics 403 is? Has it been moved?\n",
        "#Person2#: OK. Let me check on the computer. Err I'm sorry, but it says here that the class was cancelled. You should have got a notice letter about this.\n",
        "#Person1#: What? I never got it.\n",
        "#Person2#: Are you sure? It says on the computer that the letter was sent out to the students a week ago.\n",
        "#Person1#: Really? I should have got it by now. I wonder if I threw it away with all the junk mail by mistake.\n",
        "#Person2#: Well, it does happen. Err let me check something. What's your name?\n",
        "#Person1#: Woodhouse Laura Woodhouse.\n",
        "#Person2#: OK, Woodhouse. Let me see. Ah, it says here we sent it to your apartment on the Center Street.\n",
        "#Person1#: Oh, that's my old apartment. I moved out of there a little while ago.\n",
        "#Person2#: Well, I suppose you haven't changed your mailing address at the administration office.\n",
        "#Person1#: Yeah, I should have changed it in time.\n",
        "\"\"\"\n",
        "\n",
        "example_summary = \"\"\"\n",
        "Summary:\n",
        "Laura Woodhouse finds out physics is canceled but she never received the mail. #Person2# finds her mailing address is her old apartment. Laura thinks she should have changed it in time.\n",
        "\"\"\"\n",
        "\n",
        "rater_system_message = \"\"\"\n",
        "You are tasked with rating AI-generated summaries of dialogues based on the given metric.\n",
        "You will be presented a dialogue and an AI generated summary of the dialogue as the input.\n",
        "In the input, the dialogue will begin with ###Dialogue while the AI generated summary will begin with ###Summary.\n",
        "\n",
        "Evaluation criteria:\n",
        "The task is to judge the extent to which the metric is followed by the summary.\n",
        "1 - The metric is not followed at all\n",
        "2 - The metric is followed only to a limited extent\n",
        "3 - The metric is followed to a good extent\n",
        "4 - The metric is followed mostly\n",
        "5 - The metric is followed completely\n",
        "\n",
        "Metric:\n",
        "Faithfulness – The factual accuracy and verifiability of the information presented in the summary.\n",
        "Every piece of information mentioned in the summary should be directly verifiable, supported, or reasonably inferred from the dialogue.\n",
        "\n",
        "Instructions:\n",
        "1. First write down the steps that are needed to evaluate the summary as per the metric.\n",
        "2. Give a step-by-step explanation if the summary adheres to the metric considering the dialogues as the input.\n",
        "3. Next, evaluate the extent to which the metric is followed.\n",
        "4. Use the previous information to rate the summary using the evaluaton criteria and assign a score.\n",
        "5. Present your evaluation in the following JSON format:\n",
        "{\n",
        "    'step_by_step_explanation': ,\n",
        "    'score': \n",
        "}\n",
        "\"\"\"\n",
        "\n",
        "rater_model = 'gpt-4o-mini'\n",
        "\n",
        "rater_user_message = f\"\"\"\n",
        "###Dialogue\n",
        "{example_dialogue}\n",
        "\n",
        "###Summary\n",
        "{example_summary}\n",
        "\"\"\"\n",
        "\n",
        "rater_prompt = [\n",
        "    {'role': 'developer', 'content': rater_system_message},\n",
        "    {'role': 'user', 'content': rater_user_message}\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=rater_model,\n",
        "    messages=rater_prompt\n",
        ")\n",
        "\n",
        "print(\"LLM Judge Evaluation:\")\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Power Ahead!\n",
        "\n",
        "You've completed the prompt engineering fundamentals notebook. Feel free to experiment with different prompting techniques and parameters!"
      ]
    }
  ]
}.\n",
        "The final straw was when the phone's camera malfunctioned, and the repair cost was astronomical.\n",
        "I demand a full refund and an apology for this abysmal product.\n",
        "Returning it would be a relief, as this phone has become nothing but a money pit. Beware, fellow buyers!\n",
        "\"\"\"\n",
        "\n",
        "cot_prompt = [\n",
        "        {\"role\": \"developer\", \"content\": system_message},\n",
        "        {\"role\": \"user\", \"content\": customer_complaint}\n",
        "]\n",
        "\n",
        "response = client.chat.completions.create(\n",
        "    model=model_name,\n",
        "    messages=cot_prompt,\n",
        "    temperature=0\n",
        ")\n",
        "\n",
        "print(response.choices[0].message.content)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Prompt Chaining"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "customer_complaint = \"\"\"\n",
        "I am fuming with anger and regret over my purchase of the XUI890.\n",
        "First, the price tag itself was exorbitant at 1500 $, making me expect exceptional quality.\n",
        "Instead, it turned out to be a colossal disappointment.\n",
        "The additional charges to fix its constant glitches and defects drained my wallet even more.\n",
        "I spend 275 $ to get a new battery